# Récupération des données du ficher "Variables La Serve.csv"

librairies python utilisées...

In [1]:
import csv
import re
from utils import Resize
from modbusDefinition import getVarCoeff, modbusDefinition

**Varoiables globales**
  - "fileCsvName" : nom du fichier à parser  
  - "delimiter" : type de délimiteur (pour le fichier csv à parser)
  - "verbose" : activation des log de debug 

In [2]:
fileCsvName  	= "./csv_inputs/Nouvergies - La Serve/Variables La Serve.csv"
delimiter 		= ";"
verbose = True

In [3]:
### notes...
# une ligne _requête_ 'DEFAUT DEGRILLEUR' a été ajoutée manuellement au CSV pour ne pas passer de l'adresse 2xxx à 4xxx dans la même requête :   
# 					;;;     DEFAUT DEGRILLEUR:;;;;;;;;;;;;;;;;;;

## Analyse du ficgier CSV
on ne récupère que les colonnes :
  - **'Libellé'**, qui correspond 
    - au libellé du groupe (si la colonne 'Ad API' est vide)  
    - au libellé du groupe (si la colonne 'Ad API' contient une adresse)  
  - **'Ad API'**, qui contient l'adresse, et le type de variable  
  - **'Unités'**, qui contient l'unité des données à récupérer  
  - les coefs **'Val.API mini', 'Val. API maxi', 'Val. fin mini', et 'Val. fin maxi'** permettent de cacluler les coefs **'varCoeffA'** et **'varCoeffB'** à appliquer à la valeur lue.  
  

### choix des lignes de variables
Il semble que la colonne "CE"  soit un compteur de variables.   
... on ne prend comme variables que les lignes ayant un CE.  

In [4]:
csvData = {}

### a priori les premières lignes du fichier sont deq défauts, alarmes, etc. ###
reqName = "DEFAUTS_ET_ALARMES" 						# nom de requête pour les premières lignes : "DEFAUTS_ET_ALARMES"			
csvData[ reqName ] = {}								# 	on associe à cette requête une structure qui contiendra les variables

with open(fileCsvName) as csvfile:
	reader = csv.DictReader(csvfile, delimiter=delimiter)

	for row in reader:
		#print(row)

		####	récupération des colonnes utiles 	####
		if 'Libellé' in row.keys() :
			label 		= str(row['Libellé']).replace("     ","").replace(" ","_").replace(":","")
			CE 			= row['CE']					# ... il semble que la colonne "CE"  soit un compteur de variables -> on ne prend comme variables que les lignes ayant un CE

			# données utiles
			addrAPI 	= row['Ad API']				# type + adresse des variabnles
			unit 		= row['Unités']				# unité
			x1			= row['Val.API mini']		# abscisse point 1 pour la "mise à l’échelle de la variable (Ax +B)"
			x2			= row['Val. API maxi']		# abscisse point 2 pour la "mise à l’échelle de la variable (Ax +B)"
			y1			= row['Val. fin mini']		# ordonnée point 1 pour la "mise à l’échelle de la variable (Ax +B)"
			y2			= row['Val. fin maxi']		# ordonnée point 2 pour la "mise à l’échelle de la variable (Ax +B)"
		else :
			print("'Libellé' nest pas dans les colonne du csv : \n" + str(row.keys()) )
			
		# on prend commme nom de requête les 'Libellé' qui dont la colonne 'Ad API' est vide
		if addrAPI == "" :						# 'Libellé' qui dont la colonne 'Ad API' est vide
			if "@PING192.168" not in label:		# 	exception : on prend pas les "ping..." de la fin !
				reqName = label					# 	... on le prend commme nom de requête
				csvData[ reqName ] = {}			# 	on associe à cette requête une structure qui contiendra les variables
			#else : print("on ignopre la ligne de pring : " + str(reqName) + " -> " + str(label) )
		
		else :									# ligne avec un type et une adresse de définie (colonne 'Ad API' non vide)

			# ... il semble que la colonne "CE"  soit un compteur de variables
			if CE != "" :  						# on ne prend comme variables que les lignes ayant un CE non vide
				csvData[ reqName ][ label ] = { 	
					"addr"		: addrAPI	,	# type + adresse des variabnles
					"unit"		: unit		,	# unité
					"x1"  		: x1		,	# abscisse point 1 pour la "mise à l’échelle de la variable (Ax +B)"
					"y1"  		: y1		,	# abscisse point 2 pour la "mise à l’échelle de la variable (Ax +B)"
					"x2"  		: x2		,	# ordonnée point 1 pour la "mise à l’échelle de la variable (Ax +B)"
					"y2"  		: y2		,	# ordonnée point 2 pour la "mise à l’échelle de la variable (Ax +B)"
				}

# Affichage des requêtes trouvés et suppression des requêtes vides

In [5]:
emptyReqList = []
for reqName in csvData:	
	print(" ---- '" + str(reqName) + "' ---- " )
	if len(csvData[reqName]) == 0 : 
		print("   " + Resize("", 60)+ " --> requête vide : on ne la prend pas"  )
		emptyReqList.append(reqName)	# liste des requêtes vides
	else :
		for varName in csvData[reqName] :
			print("   " + Resize(varName, 60) + " --> " + str(csvData[reqName][varName]) )

# suppression des requêtes vides
for r in emptyReqList :
	csvData.pop(r)


 ---- 'DEFAUTS_ET_ALARMES' ---- 
   Défaut_mesure_de_niveau_amont_grille                         --> {'addr': '%M101', 'unit': '', 'x1': '0', 'y1': '0', 'x2': '1', 'y2': '1'}
   Défaut_mesure_de_niveau_aval_centrale                        --> {'addr': '%M103', 'unit': '', 'x1': '0', 'y1': '0', 'x2': '1', 'y2': '1'}
   Défaut_niveau_bas                                            --> {'addr': '%M104', 'unit': '', 'x1': '0', 'y1': '0', 'x2': '1', 'y2': '1'}
   Alarme_niveau_haut                                           --> {'addr': '%M105', 'unit': '', 'x1': '0', 'y1': '0', 'x2': '1', 'y2': '1'}
   Alarme_niveau_bas                                            --> {'addr': '%M112', 'unit': '', 'x1': '0', 'y1': '0', 'x2': '1', 'y2': '1'}
   Défaut_acquisition_température_centrale                      --> {'addr': '%M115', 'unit': '', 'x1': '0', 'y1': '0', 'x2': '1', 'y2': '1'}
   Défaut_acquisition_température_extérieur                     --> {'addr': '%M116', 'unit': '', 'x1': '0', 'y1': 

# défintion des constantes 

In [6]:
#(req)id		: "?"	# compteur de requêtes 			-> calculé
#(req)Name		= "?"	# nom de requête 				-> pris dans le fichier csv
ReadFctCode 	= 4
WriteFctCode 	= 0
#StartReg 		= min(addrList)							-> calculé depuis les données du fichier csv
#NbReg			= len(addrList)							-> calculé depuis les données du fichier csv
EnableReading	= 1
EnableWritting	= 2
Option1			= 0
Option2			= 0

#varIndex		= 1		# compteur de variables 		-> calculé
#varReqIndex	= 1		# index du groupe (requête)		-> calculé
#varName		= "?"	# nom de variable 				-> pris   dans le fichier csv
varType			= 8		# type de variable 				-> depuis dans le fichier csv (à vérifier)
varSigned		= 1		# signe de la variable 	-> ...	à prendre dans le fichier csv ???
varPosition		= "?"	# adresse relative				-> depuis dans le fichier csv (à vérifier)
varOption1		= ""
varOption2		= ""
varCoeffA		= 1		# gain pour la variable			-> depuis dans le fichier csv (à vérifier)
varCoeffB		= 0		# offset de la variable			-> depuis dans le fichier csv (à vérifier)
varUnit			= ""	# unité  de la variable			-> pris dans le fichier csv
varAction		= 2




## Fonction de conversion des types de données  
Les types "MW", "MD?", "M" fournis par le SCADA en même temps que l'adresse sont convertis en types "MODUS" (colonne 'varType') 
  
**TODO** :  
  - vérifier la conversion ci-dessous, et trouver à quoi correspond "MD",  
  - vérifier que l'adresse n'est pas impactée (multiplication x2 ou x4 de l'adresser suivant les types ?),   
  - vérifier que le signe n'est pas impacté (colonne MODBUS 'varSigned'),   

In [7]:
def dataTypeTranslate(wordType="MW") :
	d = {
		"MW"		: 8	, 		# /!\ TODO : à vérifier /!\
		"MD"		: 777, 		# /!\ TODO : à modifier /!\  --->  au hasard !
		"M" 		: 2	, 		# /!\ TODO : à vérifier /!\
	}

	try :
		varType = d[wordType]
	except :
		varType = str(wordType) + "???"

	return varType

# Construction du fichier modbus  

**Valeurs d'initialisation**  

2 structures permettent de choisir les valeur d'initialisations des colonnes MODBUS :  
  - pour les requêtes  : **mb.RequestsTableColumns**
  - pour les variables : **mb.VariablesTableColumns**
  
... ici, on ne fait que les affichier
_(mais il est possible de modifier ces valeurs d'init via les attributs ci-dessus)_ 

In [8]:
mb = modbusDefinition("La Serve")
mb.printReqColumns(True)
mb.printVarColumns(True)

RequestsTableColumns...
# id;Name;ReadFctCode;WriteFctCode;StartReg;NbReg;EnableReading;EnableWritting;Option1;Option2
   id                   ---> ?
   Name                 ---> ?
   ReadFctCode          ---> 3
   WriteFctCode         ---> 0
   StartReg             ---> 0
   NbReg                ---> 0
   EnableReading        ---> 1
   EnableWritting       ---> 2
   Option1              ---> 0
   Option2              ---> 0
VariablesTableColumns...
# varIndex;varReqIndex;varName;varType;varSigned;varPosition;varOption1;varOption2;varCoeffA;varCoeffB;varUnit;varAction
   varIndex             ---> ?
   varReqIndex          ---> ?
   varName              ---> ?
   varType              ---> 8
   varSigned            ---> 2
   varPosition          ---> 1
   varOption1           ---> 
   varOption2           ---> 
   varCoeffA            ---> 1
   varCoeffB            ---> 0
   varUnit              ---> 
   varAction            ---> 2


# Décodages des valeurs (trouvées dans le CSV), et appel de la fonction "add_Request_And_Variables_with_adresses()"
L'appel à la fonction "add_Request_And_Variables_with_adresses()" permet d'ajouter d'un seul bloc :   
  - la requête, et 
  - les variables qui lui sont associées,

Cela pemet aussi d'automatiser un certain nombre de calculs :
  - [id] l'indetifiant (compteur) des requêtes,  
  - [StartReg] l'adresses de début de requête,  
  - [NbReg] le comptage du nombre d'octets à lire pour la requête,  
    
  - [varIndex] l'indetifiant (compteur) des variables,  
  - [varReqIndex] l'association des variables à leur requête,  
  - [varPosition] le calcul de l'offset de la variable par rapport àson adresse de début,  
  
_Note : la requête peut alors n'être définie que par son nom [Name], le reste des champs étant calculé ou figés par une valeur d'init._

## Décodage "type/adresse"
La colonne 'addr' du CSV, regroupe le type de variable et son adressse. On fait par un regex qui sépare les 2 :  
  - type = les premières lettres,  
  - adresse = les chiffres qui suivent.  
Le type est ensuite recodée au format MODBUS via la fonction 'dataTypeTranslate' (voir ci-dessus).  
  
**Principe du décodage de l'adresse (Regex)**  
Utilsation du regex **%([A-Z]+)([0-9]+)** :
  - '%' indique qu'on veut être en début de mot,  
  - ([A-Z]+) est un chaine de lettres qui correspond au type de données,    
  - ([0-9]+) est un chaine de chiffres qui correspond à l'adresse,  
  
... _on aurait pu pourrait utiliser \w+\d+ ?_  


## Calcul de l'interpolation linéaire (coefs A & B)  
Les colonnes du CSV ont été renommées en abscisses et ordonnées de 2 points dont on fait l'interpolation linéaire via la fonction 'getVarCoeff()' :  
  - x1	<= 'Val.API mini'   
  - x2	<= 'Val. API maxi'  
  - y1	<= 'Val. fin mini'  
  - y2	<= 'Val. fin maxi'  
	
... ce qui permet d'avoir les colonnes 'varCoeffA' et 'varCoeffB' de la definition MODBUS.  
  
## L'unité  
Dans la colonne MODBUS 'varUnit', on reprend telle quelle la colonne CSV 'unit'.  


In [9]:
for reqName in csvData:	
	#print(" ---- '" + str(reqName) + "' ---- " )
	V = {}

	for varName in csvData[reqName] :
		#print("   " + Resize(varName, 60) + " --> " + str(csvData[reqName][varName]) )
		
		### 	décodage des variables	 ###
		# on utilise un regex pour trouver l'adresse de la requête
		regex = '%([A-Z]+)([0-9]+)'
		reAddr = re.match(regex, csvData[reqName][varName]['addr'])			# adresse -> type de données + ligne

		if reAddr :     	# regex OK ---> adr. trouvée ...
			wordType    	=     reAddr.groups()[0]    	# type (W, WM, WD, ...)
			wordAddr    	= int(reAddr.groups()[1])    	# adresse

			# coefs A, B à partir des valeurs min/max 
			(A, B) = getVarCoeff(	csvData[reqName][varName]['x1'],						# coefs A & B
									csvData[reqName][varName]['y1'],
									csvData[reqName][varName]['x2'],
									csvData[reqName][varName]['y2']	)

			# construction d'un dictionnaire décrivant la requête (avec les noms de colonne 'modbus')
			newVar = {
				#'varIndex' 	: calculé automatiquement
				#'varReqIndex'	: calculé automatiquement
				'varName'		: varName									,
				'varType'		: dataTypeTranslate(wordType)				, 	# TODO : vérifier la conversion lettre -> chiffre ???
				#'varSigned'	: en dure (VariablesTableColumns)
				#'varPosition'	: calculé automatiquement à partir de l'adresse 'wordAddr'
				#'varOption1'	: en dure (VariablesTableColumns)
				#'varOption2'	: en dure (VariablesTableColumns)				
				'varCoeffA'		: A											,	# mise à l’échelle de la variable (Ax +B)
				'varCoeffB'		: B											,	# mise à l’échelle de la variable (Ax +B)
				'varUnit'		: str(csvData[reqName][varName]['unit'])	,
				#'varAction'	: en dure (VariablesTableColumns)			
				'debug'			: csvData[reqName]
			}

			####	détection des adresses en doubles (dans la définiton des variables)	###
			if wordAddr not in V :
				V[wordAddr] = newVar
			else :
				print("\n")
				print("/!\\ Il exite déjà une variabble à l'adresse " + str(wordAddr) + "... /!\\")
				print(V[wordAddr])
				print(newVar)
				raise TypeError("/!\\ Duplication d'adresse de variables /!\\")


	if reqName :	# suppression des ligne où le nom de requête n'est pas défini...

		if 1 :		# debug
			print("req: " +Resize(reqName) + "...")
			for a in V : 
				print("      @" + Resize(a, 4) + ": " +	mb.strValues(V[a]) )		# affichage des valeurs d'un dictionnaire définissant une variable 

		# ajout de la requêtes (R) et des varibles associées indexées par les adresse (V) 
		mb.add_Request_And_Variables_with_adresses(reqName, V, False)

req: DEFAUTS_ET_ALARMES  ...
      @101 : varName=Défaut_mesure_de_niveau_amont_grille, varType=2, varCoeffA=1.0, varCoeffB=0.0
      @103 : varName=Défaut_mesure_de_niveau_aval_centrale, varType=2, varCoeffA=1.0, varCoeffB=0.0
      @104 : varName=Défaut_niveau_bas, varType=2, varCoeffA=1.0, varCoeffB=0.0
      @105 : varName=Alarme_niveau_haut, varType=2, varCoeffA=1.0, varCoeffB=0.0
      @112 : varName=Alarme_niveau_bas, varType=2, varCoeffA=1.0, varCoeffB=0.0
      @115 : varName=Défaut_acquisition_température_centrale, varType=2, varCoeffA=1.0, varCoeffB=0.0
      @116 : varName=Défaut_acquisition_température_extérieur, varType=2, varCoeffA=1.0, varCoeffB=0.0
      @117 : varName=Protection_de_découplage, varType=2, varCoeffA=1.0, varCoeffB=0.0
      @118 : varName=Protection_de_découplage_longue, varType=2, varCoeffA=1.0, varCoeffB=0.0
      @121 : varName=Arrêt_d'urgence_général, varType=2, varCoeffA=1.0, varCoeffB=0.0
      @128 : varName=Protection_C13-100, varType=2, varCoef

**décodage de l'adresse**
  -  utilsation d'un regex '%(\w)(\d+)':
	- '%' ets le début du mot   
	- ([A-Z]+) est un chaine de lettres qui correspond au type de données    
	- ([0-9]+) est un chaine de chiffres qui correspond à l'adresse,  
	 ... on pourrait utiliser (\d+) ?  


# Fichier Modbus

In [10]:
print("------------------------------------------------ Requêtes  ------------------------------------------------")
mb.printRequest('all')

------------------------------------------------ Requêtes  ------------------------------------------------
req1     --- DEFAUTS_ET_ALARMES                  ---> '1;DEFAUTS_ET_ALARMES;3;0;101;59;1;2;0;0'
req2     --- ETAT_GROUPE                         ---> '2;ETAT_GROUPE;3;0;2001;143;1;2;0;0'
req3     --- DEFAUT_DEGRILLEUR                   ---> '3;DEFAUT_DEGRILLEUR;3;0;4007;28;1;2;0;0'
 ----------------------------------------- 
   id                Name  ReadFctCode  WriteFctCode  StartReg  NbReg  EnableReading  EnableWritting  Option1  Option2
0   1  DEFAUTS_ET_ALARMES            3             0       101     59              1               2        0        0
1   2         ETAT_GROUPE            3             0      2001    143              1               2        0        0
2   3   DEFAUT_DEGRILLEUR            3             0      4007     28              1               2        0        0


In [11]:
print("------------------------------------------------ Variables ------------------------------------------------")
mb.printVariables('all')

------------------------------------------------ Variables ------------------------------------------------
----------------------------------------------------------------------------------
req 1 (DEFAUTS_ET_ALARMES) : variables... 
   0;1;Défaut_mesure_de_niveau_amont_grille;2;2;1;;;1.0;0.0;;2
   1;1;reserve;2;2;2;;;1;0;;0
   2;1;Défaut_mesure_de_niveau_aval_centrale;2;2;3;;;1.0;0.0;;2
   3;1;Défaut_niveau_bas;2;2;4;;;1.0;0.0;;2
   4;1;Alarme_niveau_haut;2;2;5;;;1.0;0.0;;2
   5;1;reserve;2;2;6;;;1;0;;0
   6;1;reserve;2;2;7;;;1;0;;0
   7;1;reserve;2;2;8;;;1;0;;0
   8;1;reserve;2;2;9;;;1;0;;0
   9;1;reserve;2;2;10;;;1;0;;0
   10;1;reserve;2;2;11;;;1;0;;0
   11;1;Alarme_niveau_bas;2;2;12;;;1.0;0.0;;2
   12;1;reserve;2;2;13;;;1;0;;0
   13;1;reserve;2;2;14;;;1;0;;0
   14;1;Défaut_acquisition_température_centrale;2;2;15;;;1.0;0.0;;2
   15;1;Défaut_acquisition_température_extérieur;2;2;16;;;1.0;0.0;;2
   16;1;Protection_de_découplage;2;2;17;;;1.0;0.0;;2
   17;1;Protection_de_découplage_long

## Création du fichier

In [12]:
mb.modbusDefFile(outputDir= "./out/Nouvergies/", fileName = "modbus_La_Serve.ini", verbose = True)

>>> création du répertoire './out/Nouvergies/'

 --- fichier ./out/Nouvergies/modbus_La_Serve.json créé ---- 
#Based on La Serve collected values

# Description of fields
# id;Name;ReadFctCode;WriteFctCode;StartReg;NbReg;EnableReading;EnableWritting;Option1;Option2
Modbus_RequestsTables={
1;DEFAUTS_ET_ALARMES;3;0;101;59;1;2;0;0
2;ETAT_GROUPE;3;0;2001;143;1;2;0;0
3;DEFAUT_DEGRILLEUR;3;0;4007;28;1;2;0;0
}



# Description of fields
# varIndex;varReqIndex;varName;varType;varSigned;varPosition;varOption1;varOption2;varCoeffA;varCoeffB;varUnit;varAction
#
Modbus_VariablesTables={
0;1;Défaut_mesure_de_niveau_amont_grille;2;2;1;;;1.0;0.0;;2
1;1;reserve;2;2;2;;;1;0;;0
2;1;Défaut_mesure_de_niveau_aval_centrale;2;2;3;;;1.0;0.0;;2
3;1;Défaut_niveau_bas;2;2;4;;;1.0;0.0;;2
4;1;Alarme_niveau_haut;2;2;5;;;1.0;0.0;;2
5;1;reserve;2;2;6;;;1;0;;0
6;1;reserve;2;2;7;;;1;0;;0
7;1;reserve;2;2;8;;;1;0;;0
8;1;reserve;2;2;9;;;1;0;;0
9;1;reserve;2;2;10;;;1;0;;0
10;1;reserve;2;2;11;;;1;0;;0
11;1;Alarme_niveau_bas